In [1]:
using Revise

using Test
using Random

using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
import TCIAlgorithms as TCIA
using ITensors
ITensors.disable_warn_order()
import QuanticsGrids: DiscretizedGrid, quantics_to_origcoord

In [13]:
_getindex(x, indices) = ntuple(i->x[indices[i]], length(indices))

function _contract(
    a::AbstractArray{T1,N1},
    b::AbstractArray{T2,N2},
    idx_a::NTuple{n1,Int},
    idx_b::NTuple{n2,Int}
    ) where {T1, T2, N1, N2, n1, n2}
    length(idx_a) == length(idx_b) || error("length(idx_a) != length(idx_b)")
    # check if idx_a contains only unique elements
    length(unique(idx_a)) == length(idx_a) || error("idx_a contains duplicate elements")
    # check if idx_b contains only unique elements
    length(unique(idx_b)) == length(idx_b) || error("idx_b contains duplicate elements")
    # check if idx_a and idx_b are subsets of 1:N1 and 1:N2
    all(1 <= idx <= N1 for idx in idx_a) || error("idx_a contains elements out of range")
    all(1 <= idx <= N2 for idx in idx_b) || error("idx_b contains elements out of range")

    rest_idx_a = setdiff(1:N1, idx_a)
    rest_idx_b = setdiff(1:N2, idx_b)

    amat = reshape(permutedims(a, (rest_idx_a..., idx_a...)), prod(_getindex(size(a), rest_idx_a)), prod(_getindex(size(a), idx_a)))
    bmat = reshape(permutedims(b, (idx_b..., rest_idx_b...)), prod(_getindex(size(b), idx_b)), prod(_getindex(size(b), rest_idx_b)))

    return reshape(amat * bmat, size(a)[rest_idx_a...]..., size(b)[rest_idx_b]...)
end

_contract (generic function with 1 method)

In [16]:
a = rand(2, 3, 4)
b = rand(2, 5, 4)
ab = _contract(a, b, (1, 3), (1, 3))

3×5 Matrix{Float64}:
 1.9065   3.86227  2.6992   2.38542  2.32756
 1.35216  2.76803  1.49784  1.68708  1.66962
 1.56337  2.81816  2.12559  1.76629  1.60034

In [9]:
Tuple([1,2])

(1, 2)

In [8]:
fxy(x, y) = exp(-x^2 - y^2)

R = 40
grid = DiscretizedGrid{2}(R, (0, 0), (25, 25))
localdims = fill(4, R)

qf = x -> fxy(quantics_to_origcoord(grid, x)...)

tol = 1e-7

pordering = TCIA.PatchOrdering(collect(1:R))

creator = TCIA.TCI2PatchCreator(
    ComplexF64,
    qf,
    localdims;
    maxbonddim=10,
    rtol=tol,
    verbosity=1,
    ntry=10,
)

tree = TCIA.adaptivepartion(creator, pordering; verbosity=1, maxnleaves=1000)

iteration = 1, rank = 9, error= 8.622563023151864e-8, maxsamplevalue= 1.0, nglobalpivot=5
iteration = 2, rank = 15, error= 0.00028610945115262036, maxsamplevalue= 1.0, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration
iteration = 3, rank = 14, error= 0.0002688135238189059, maxsamplevalue= 1.0, nglobalpivot=5
  Rejected 3 global pivots added in the previous iteration
iteration = 4, rank = 15, error= 0.0002660690834284218, maxsamplevalue= 1.0, nglobalpivot=5
Creating a task for [1] ...
Creating a task for [2] ...
Creating a task for [3] ...
Creating a task for [4] ...
iteration = 1, rank = 9, error= 8.622563023151864e-8, maxsamplevalue= 1.0, nglobalpivot=5
iteration = 1, rank = 0, error= 1.3851193699226017e-68, maxsamplevalue= 1.3851193699226017e-68, nglobalpivot=0
  Rejected 1 global pivots added in the previous iteration
iteration = 2, rank = 0, error= 0.0, maxsamplevalue= 1.3851193699226017e-68, nglobalpivot=0
iteration = 2, rank = 14, error= 0.0002877448285110

ErrorException: Error in creating a patch for [2]: TCI has zero rank, maxsamplevalue: 1.3851193699226017e-68, tolerance: (1.0e-7)
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] _crossinterpolate2(::Type{ComplexF64}, f::Function, localdims::Vector{Int64}, initialpivots::Vector{Vector{Int64}}, tolerance::Float64; maxbonddim::Int64, verbosity::Int64)
    @ TCIAlgorithms ~/git/TCIAlgorithms.jl/src/divideandconquer/adaptivepartitioning.jl:312
  [3] _crossinterpolate2
    @ ~/git/TCIAlgorithms.jl/src/divideandconquer/adaptivepartitioning.jl:286 [inlined]
  [4] createpatch(obj::TCIAlgorithms.TCI2PatchCreator{ComplexF64}, pordering::TCIAlgorithms.PatchOrdering, prefix::Vector{Vector{Int64}})
    @ TCIAlgorithms ~/git/TCIAlgorithms.jl/src/divideandconquer/adaptivepartitioning.jl:336
  [5] (::TCIAlgorithms.var"#126#130"{TCIAlgorithms.TCI2PatchCreator{ComplexF64}, TCIAlgorithms.PatchOrdering, Vector{Int64}})()
    @ TCIAlgorithms ~/.julia/juliaup/julia-1.10.2+0.aarch64.apple.da

In [3]:
using QuanticsTCI

iteration = 3, rank = 0, error= 0.0, maxsamplevalue= 1.918555668934785e-136, nglobalpivot=0


In [4]:
qtci, ranks, errors = quanticscrossinterpolate(Float64, fxy, grid; tolerance=tol)

MethodError: MethodError: no method matching optimize!(::TensorCrossInterpolation.TensorCI2{Float64}, ::TensorCrossInterpolation.CachedFunction{Float64, UInt128}; nsearchglobalpivot::Int64, tolerance::Float64, strictlynested::Bool)

Closest candidates are:
  optimize!(::TensorCrossInterpolation.TensorCI2{ValueType}, ::Any; tolerance, pivottolerance, maxbonddim, maxiter, sweepstrategy, pivotsearch, verbosity, loginterval, normalizeerror, ncheckhistory, maxnglobalpivot, nsearchglobalpivot, tolmarginglobalsearch, partialnesting) where ValueType got unsupported keyword argument "strictlynested"
   @ TensorCrossInterpolation ~/.julia/packages/TensorCrossInterpolation/K5op2/src/tensorci2.jl:637
